# Import Libraries

In [2]:
import os
import gc
from pprint import pprint

import numpy as np

import matplotlib.pyplot as plt
# import matplotlib
from matplotlib.lines import Line2D
from matplotlib.legend import Legend

import caesar

import h5py
import yt
import pynbody as pnb

In [3]:
gc.isenabled()

True

In [4]:
def euclidean_distance(a, b):
    # assert np.shape(a)[np.ndim(a)-1] == np.shape(b)[np.ndim(b)-1], f'Axis of a and b for calculationg are different lengths'
    assert np.shape(a) == np.shape(b), f'Shapes of a and b are different'
    return(np.sqrt(np.sum((a-b)**2, axis=np.ndim(a)-1, keepdims=True)))

# Load snapshot with h5py, yt, and pynbody

In [5]:
snap_file = '/scratch/b/babul/wcui/HYENAS/Level0/halo_3224/snap_halo_3224_151.hdf5'
# snap_file = '/project/b/babul/renierht/Kobayashi_N512L50/snapshot_151.hdf5'
# a = 1
# h = 0.68

In [6]:
snap_dict = {
    'h5py':{
        'snap':h5py.File(snap_file, 'r'),
    },
    'yt':{
        'snap':yt.load(snap_file),
    },
    'pnb':{
        'snap':pnb.load(snap_file),
    },
}

yt : [INFO     ] 2025-09-03 17:55:49,515 ComovingIntegrationOn does not exist, falling back to OmegaLambda
yt : [INFO     ] 2025-09-03 17:55:49,543 Calculating time from 1.000e+00 to be 4.375e+17 seconds
yt : [INFO     ] 2025-09-03 17:55:49,544 Assuming length units are in kpc/h (comoving)
yt : [INFO     ] 2025-09-03 17:55:49,643 Parameters: current_time              = 4.3748565942961094e+17 s
yt : [INFO     ] 2025-09-03 17:55:49,644 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2025-09-03 17:55:49,644 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2025-09-03 17:55:49,645 Parameters: domain_right_edge         = [200000. 200000. 200000.]
yt : [INFO     ] 2025-09-03 17:55:49,645 Parameters: cosmological_simulation   = True
yt : [INFO     ] 2025-09-03 17:55:49,646 Parameters: current_redshift          = 4.440892098500626e-16
yt : [INFO     ] 2025-09-03 17:55:49,646 Parameters: omega_lambda              = 0.7
yt : [INFO     ] 2025-09-03 17:55:49,647 

In [7]:
## Correct pynbody's reading of gizmo units and then set to physical units
snap_dict['pnb']['snap'].set_units_system(velocity='km a**0.5 s**-1', distance='kpc a h**-1', mass='1e10 Msol h**-1')
snap_dict['pnb']['snap'].physical_units()

In [8]:
h = snap_dict['pnb']['snap'].properties['h']
a = snap_dict['pnb']['snap'].properties['a']

In [9]:
a

np.float64(0.9999999999999996)

In [10]:
h

np.float64(0.68)

# Get parameters (e.g. minpotpos, r500c) of desired halo for filter from caesar file

In [11]:
caesar_file = '/scratch/b/babul/wcui/HYENAS/Level0/halo_3224/Groups/Caesar_halo_3224_151.hdf5'
# caesar_file = '/project/b/babul/renierht/Kobayashi_N512L50/caesar_151.hdf5'

In [12]:
snap_dict['caesar'] = {
    'obj':caesar.load(caesar_file),
}

yt : [INFO     ] 2025-09-03 17:55:49,934 Opening /scratch/b/babul/wcui/HYENAS/Level0/halo_3224/Groups/Caesar_halo_3224_151.hdf5
yt : [INFO     ] 2025-09-03 17:55:49,967 Found 1021 halos
yt : [INFO     ] 2025-09-03 17:55:49,972 Found 320 galaxies


In [13]:
halo_id = 3
halo = snap_dict['caesar']['obj'].halos[halo_id]
snap_dict['caesar']['minpotpos'] = halo.minpotpos.in_units('Mpc')
snap_dict['caesar']['r500c'] = halo.virial_quantities['r500c'].in_units('Mpc')
snap_dict['caesar']['r200c'] = halo.virial_quantities['r200c'].in_units('Mpc')

In [14]:
pprint(snap_dict)

{'caesar': {'minpotpos': unyt_array([151.6479 , 141.85269, 143.66356], dtype=float32, units='Mpc'),
            'obj': <caesar.loader.CAESAR object at 0x151a5e48c830>,
            'r200c': unyt_quantity(0.48152631, 'Mpc'),
            'r500c': unyt_quantity(0.322095, 'Mpc')},
 'h5py': {'snap': <HDF5 file "snap_halo_3224_151.hdf5" (mode r)>},
 'pnb': {'snap': <GadgetHDFSnap "/scratch/b/babul/wcui/HYENAS/Level0/halo_3224/snap_halo_3224_151.hdf5" len=7504320>},
 'yt': {'snap': GizmoDataset: /scratch/b/babul/wcui/HYENAS/Level0/halo_3224/snap_halo_3224_151.hdf5}}


# Make filter (e.g. sphere) and get number of particles in it

In [15]:
sphere_centre = snap_dict['caesar']['minpotpos']
sphere_radius = snap_dict['caesar']['r500c']

In [16]:
sphere_centre

unyt_array([151.6479 , 141.85269, 143.66356], dtype=float32, units='Mpc')

In [17]:
sphere_radius

unyt_quantity(0.322095, 'Mpc')

## h5py

### Manual Calculation

In [233]:
## h5py
snap_dict['h5py']['snap'].keys()

<KeysViewHDF5 ['Header', 'PartType0', 'PartType1', 'PartType2', 'PartType4', 'PartType5']>

In [234]:
snap_dict['h5py']['snap']['Header'].keys()

<KeysViewHDF5 []>

In [235]:
snap_dict['h5py']['snap']['PartType0'].keys()

<KeysViewHDF5 ['AccKey', 'Coordinates', 'DelayTime', 'Density', 'Dust_Masses', 'Dust_Metallicity', 'ElectronAbundance', 'FractionH2', 'GrackleHI', 'GrackleHII', 'GrackleHM', 'GrackleHeI', 'GrackleHeII', 'GrackleHeIII', 'HaloID', 'ID_Generations', 'InternalEnergy', 'Masses', 'Metallicity', 'NWindLaunches', 'NeutralHydrogenAbundance', 'ParticleIDs', 'Potential', 'Sigma', 'SmoothingLength', 'StarFormationRate', 'Velocities']>

In [236]:
snap_dict['h5py']['snap']['PartType0']['Coordinates'][:]

array([[ 97852.33 ,  97187.59 ,  97636.16 ],
       [ 97852.6  ,  97187.15 ,  97635.56 ],
       [ 97852.4  ,  97187.26 ,  97636.055],
       ...,
       [104366.766,  89036.48 , 108662.88 ],
       [104364.69 ,  89036.89 , 108662.11 ],
       [104364.05 ,  89035.94 , 108662.875]],
      shape=(2243885, 3), dtype=float32)

In [237]:
h5py_coords_Mpc = snap_dict['h5py']['snap']['PartType0']['Coordinates'][:]/a/h/1000

In [238]:
h5py_coords_Mpc

array([[143.90048254, 142.92293199, 143.58258272],
       [143.90088465, 142.92227711, 143.58170956],
       [143.90058594, 142.92243796, 143.58243336],
       ...,
       [153.48053768, 130.93599494, 159.79835708],
       [153.47748162, 130.93660386, 159.79721967],
       [153.47653952, 130.93520221, 159.79834559]], shape=(2243885, 3))

In [239]:
len(h5py_coords_Mpc)

2243885

In [240]:
h5py_sphere_centre = np.repeat(np.array([sphere_centre]), repeats=len(h5py_coords_Mpc), axis=0)

In [241]:
h5py_sphere_centre

array([[151.6479 , 141.85269, 143.66356],
       [151.6479 , 141.85269, 143.66356],
       [151.6479 , 141.85269, 143.66356],
       ...,
       [151.6479 , 141.85269, 143.66356],
       [151.6479 , 141.85269, 143.66356],
       [151.6479 , 141.85269, 143.66356]],
      shape=(2243885, 3), dtype=float32)

In [242]:
h5py_radii_Mpc = euclidean_distance(h5py_coords_Mpc, h5py_sphere_centre)

In [243]:
np.shape(h5py_radii_Mpc)

(2243885, 1)

In [244]:
h5py_radii_Mpc

array([[ 7.82141306],
       [ 7.82093425],
       [ 7.8212446 ],
       ...,
       [19.56692427],
       [19.56536064],
       [19.56698304]], shape=(2243885, 1))

In [245]:
h5py_index_within_sphere = np.where(h5py_radii_Mpc <= sphere_radius)

In [246]:
h5py_index_within_sphere

(array([   5186,    5187,   27145, ..., 2234081, 2234082, 2234083],
       shape=(8006,)),
 array([0, 0, 0, ..., 0, 0, 0], shape=(8006,)))

In [247]:
h5py_gas_within_sphere_coords = h5py_radii_Mpc[h5py_radii_Mpc<=sphere_radius]

In [248]:
h5py_gas_within_sphere_coords

array([0.02533925, 0.03250073, 0.22814684, ..., 0.16704622, 0.16657877,
       0.16733172], shape=(8006,))

In [249]:
len(h5py_gas_within_sphere_coords)

8006

## yt

### Calculation with yt.snap.sphere

In [18]:
snap_dict['yt']['snap']

GizmoDataset: /scratch/b/babul/wcui/HYENAS/Level0/halo_3224/snap_halo_3224_151.hdf5

In [19]:
yt_data = snap_dict['yt']['snap'].all_data()

yt : [INFO     ] 2025-09-03 17:56:17,204 Allocating for 7.504e+06 particles
Loading particle index: 100%|██████████| 10/10 [00:00<00:00, 2943.58it/s]


In [26]:
yt_data['PartType5', 'Masses']

unyt_array([8.27045471e-04, 1.51811186e-02, 1.05471723e-03, 1.42016646e-03,
       1.06185197e-03, 1.03664340e-03, 1.33558293e-03, 7.68827740e-04,
       7.89033889e-04, 7.43292749e-01, 7.02865946e-04, 1.22847967e-03,
       1.08481897e-02, 3.99624556e-03, 1.13208930e-03, 1.50555116e-03,
       9.66393389e-04, 7.24619080e-04, 2.53487611e-03, 3.59583795e-02,
       1.09760137e-03, 1.10788341e-03, 7.45392404e-04, 1.33570540e-03,
       9.85587299e-01, 1.02062372e-03, 7.57080736e-04, 9.69745801e-04,
       7.66557001e-04, 9.47789755e-04, 2.35259151e+00, 1.22257636e-03,
       9.88349668e-04, 2.69604661e-02, 8.00211099e-04, 7.43480865e-04,
       9.88212763e-04, 1.29100133e-03, 1.07191980e-03, 9.88511741e-02,
       3.07609499e-01, 1.03319960e-03, 1.01668690e-03, 8.08931247e-04,
       8.44379738e-02, 8.48093652e-04, 1.07707526e-03, 1.40519033e-03,
       1.13021233e-03, 1.67907367e-03, 1.04686455e-03, 1.17506343e-03,
       8.49225791e-04, 7.71244348e-04, 8.26358621e-04, 9.70321358e-04,
 

In [21]:
yt_sphere = snap_dict['yt']['snap'].sphere(sphere_centre.in_units('kpc'), sphere_radius.in_units('kpc'))

In [22]:
yt_sphere

YTSphere (snap_halo_3224_151): , center=[4.67936553e+26 4.37711660e+26 4.43299452e+26] cm, radius=9.938813242067557e+23 cm

In [23]:
yt_sphere.keys()

dict_keys([])

In [24]:
yt_sphere['PartType5', 'Coordinates']#.in_units('Mpc')

unyt_array([[103135.734375 ,  96453.671875 ,  97664.140625 ],
       [103180.890625 ,  96401.65625  ,  97711.2734375],
       [103117.4921875,  96382.1484375,  97799.75     ],
       [103120.5625   ,  96459.8203125,  97691.21875  ],
       [103094.9453125,  96421.3203125,  97778.2421875],
       [103064.96875  ,  96473.2890625,  97708.4453125],
       [103122.1328125,  96456.71875  ,  97741.6953125],
       [103145.7578125,  96384.609375 ,  97811.6640625],
       [103116.34375  ,  96450.3359375,  97736.328125 ],
       [103130.6484375,  96375.0703125,  97791.3984375],
       [103110.6796875,  96461.546875 ,  97670.65625  ],
       [103115.1484375,  96463.453125 ,  97655.234375 ],
       [103149.9609375,  96466.3984375,  97548.7890625],
       [103191.1328125,  96458.015625 ,  97650.2265625],
       [103124.1171875,  96452.484375 ,  97690.828125 ],
       [103124.4296875,  96395.953125 ,  97801.65625  ],
       [103121.1015625,  96393.2890625,  97785.140625 ],
       [103124.515625 ,  9

In [313]:
yt_sphere['PartType0', 'Coordinates'].in_units('kpccm/h')

unyt_array([[103124.625    ,  96474.59375  ,  97683.3125   ],
       [103120.1875   ,  96477.9375   ,  97678.5546875],
       [103085.3671875,  96378.       ,  97818.234375 ],
       ...,
       [103188.5078125,  96428.578125 ,  97776.7265625],
       [103188.9140625,  96429.109375 ,  97776.171875 ],
       [103188.8125   ,  96428.75     ,  97776.8046875]], shape=(12175, 3), units='kpccm/h')

In [314]:
yt_sphere['PartType0', 'Coordinates'].in_units('Mpc')

unyt_array([[151.65386029, 141.87440257, 143.65193015],
       [151.64733456, 141.87931985, 143.64493336],
       [151.59612822, 141.73235294, 143.85034467],
       ...,
       [151.74780561, 141.80673254, 143.78930377],
       [151.74840303, 141.80751379, 143.78848805],
       [151.74825368, 141.80698529, 143.78941866]], shape=(12175, 3), units='Mpc')

In [315]:
yt_sphere['PartType0', 'Coordinates']/a/h/1000

unyt_array([[151.65386029, 141.87440257, 143.65193015],
       [151.64733456, 141.87931985, 143.64493336],
       [151.59612822, 141.73235294, 143.85034467],
       ...,
       [151.74780561, 141.80673254, 143.78930377],
       [151.74840303, 141.80751379, 143.78848805],
       [151.74825368, 141.80698529, 143.78941866]], shape=(12175, 3), units='code_length')

In [316]:
yt_sphere.quantities.keys()

dict_keys(['WeightedAverageQuantity', 'TotalQuantity', 'TotalMass', 'CenterOfMass', 'BulkVelocity', 'WeightedStandardDeviation', 'AngularMomentumVector', 'Extrema', 'SampleAtMaxFieldValues', 'MaxLocation', 'SampleAtMinFieldValues', 'MinLocation', 'SpinParameter'])

In [317]:
yt_sphere.quantities.total_mass().in_units('Msun')

unyt_array([2.36157195e+11, 8.82692316e+12], 'Msun')

In [318]:
yt_sphere.quantities['TotalMass']().in_units('Msun')

unyt_array([2.36157195e+11, 8.82692316e+12], 'Msun')

In [25]:
yt_sphere.quantities.total_quantity(('PartType5', 'ones'))

YTFieldNotFound: Could not find field ('PartType5', 'ones') in snap_halo_3224_151.
Did you mean:
	('PartType0', 'ones')
	('gas', 'ones')
	('index', 'ones')

In [320]:
yt_sphere.quantities.total_quantity(('gas', 'ones'))

unyt_quantity(12175., 'dimensionless')

In [321]:
len(yt_sphere['PartType0', 'ones'])

12175

In [27]:
len(yt_sphere['PartType5', 'Masses'])

40

In [322]:
len(yt_sphere['gas', 'ones'])

12175

### Manual Calculation

In [28]:
yt_sphere_centre = np.repeat(np.array([sphere_centre]), repeats=len(yt_data['PartType5', 'Coordinates']), axis=0)

In [29]:
yt_sphere_centre

array([[151.6479 , 141.85269, 143.66356],
       [151.6479 , 141.85269, 143.66356],
       [151.6479 , 141.85269, 143.66356],
       ...,
       [151.6479 , 141.85269, 143.66356],
       [151.6479 , 141.85269, 143.66356],
       [151.6479 , 141.85269, 143.66356]], shape=(617, 3), dtype=float32)

In [30]:
yt_radii_Mpc = euclidean_distance(yt_data['PartType5', 'Coordinates'].in_units('Mpc').value, yt_sphere_centre)

In [31]:
np.shape(yt_radii_Mpc)

(617, 1)

In [32]:
yt_radii_Mpc

array([[7.82167291e+00],
       [6.41906881e+00],
       [1.34995259e+01],
       [1.33912253e+01],
       [1.29315894e+01],
       [1.37135073e+01],
       [1.27012591e+01],
       [4.65285074e-02],
       [1.26713288e-01],
       [1.96326094e-01],
       [1.26995518e+01],
       [1.06716668e+01],
       [1.58296463e+01],
       [9.05584267e+00],
       [8.12495673e+00],
       [9.24879550e-01],
       [1.26933347e+01],
       [1.27074689e+01],
       [8.38910891e+00],
       [1.63700520e+01],
       [1.52670917e+01],
       [1.49643922e+01],
       [1.48885205e+01],
       [1.32856633e+01],
       [5.03925249e+00],
       [1.27519456e+01],
       [1.27703698e+01],
       [1.15452689e+01],
       [1.62631001e+01],
       [1.62619948e+01],
       [1.12687781e+01],
       [4.49785202e+00],
       [1.06959379e+01],
       [1.93846821e+01],
       [1.44850303e+01],
       [4.79902189e+00],
       [1.21849410e+01],
       [1.22087295e+01],
       [1.20337328e+01],
       [1.02546428e+01],


In [33]:
yt_index_within_sphere = np.where(yt_radii_Mpc <= sphere_radius)

In [34]:
yt_index_within_sphere

(array([  7,   8,   9,  60,  86,  87,  88, 117, 141, 188, 190, 191, 193,
        194, 265, 348, 349, 350, 362, 363, 364, 365, 377, 378, 446, 499,
        500, 501, 503, 504, 505, 506, 523, 524, 540, 546, 558, 610, 611,
        612]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [35]:
yt_gas_within_sphere_coords = yt_radii_Mpc[yt_radii_Mpc<=sphere_radius]

In [36]:
yt_gas_within_sphere_coords

array([4.65285074e-02, 1.26713288e-01, 1.96326094e-01, 2.28865997e-05,
       1.44931025e-01, 8.78647565e-02, 7.44044771e-02, 2.12086908e-01,
       6.80737155e-02, 1.93545544e-01, 3.36543836e-02, 5.37828722e-02,
       2.14087095e-01, 1.20033341e-01, 1.20074536e-02, 1.87702053e-01,
       1.69271564e-01, 1.60110933e-01, 1.23065600e-01, 1.47351447e-01,
       9.47500186e-02, 1.06462722e-01, 1.33632815e-01, 2.77827051e-01,
       2.03859659e-01, 1.83664520e-01, 7.42650178e-02, 6.67588957e-02,
       1.41581065e-01, 1.34458250e-01, 1.36753826e-01, 1.32221292e-01,
       7.79440054e-02, 1.01599021e-01, 5.21799864e-02, 2.65067012e-02,
       1.98877283e-01, 1.44824618e-01, 1.80963358e-01, 1.66247939e-01])

In [37]:
len(yt_gas_within_sphere_coords)

40

## pynbody

### Calculation with pynbody.filt.Sphere

In [277]:
snap_dict['pnb']['snap']

<GadgetHDFSnap "/scratch/b/babul/wcui/HYENAS/Level0/halo_3224/snap_halo_3224_151.hdf5" len=7504320>

In [278]:
snap_dict['pnb']['snap'].gas

<FamilySubSnap "/scratch/b/babul/wcui/HYENAS/Level0/halo_3224/snap_halo_3224_151.hdf5::gas" len=2243885>

In [279]:
snap_dict['pnb']['snap'].gas['pos']

SimArray([[143900.48, 142922.92, 143582.58],
          [143900.88, 142922.28, 143581.7 ],
          [143900.58, 142922.44, 143582.44],
          ...,
          [153480.53, 130935.99, 159798.36],
          [153477.48, 130936.6 , 159797.22],
          [153476.53, 130935.2 , 159798.34]],
         shape=(2243885, 3), dtype=float32, 'kpc')

In [280]:
pnb_sphere_centre = pnb.array.SimArray(sphere_centre, units='Mpc').in_units('kpc')
pnb_sphere_radius = pnb.array.SimArray(sphere_radius, units='Mpc').in_units('kpc')

In [281]:
# pnb_sphere_centre = pnb.array.SimArray(sphere_centre.in_units('Mpc*a/h'), units='Mpc a h**-1').in_units('kpc a h**-1')
# pnb_sphere_radius = pnb.array.SimArray(sphere_radius.in_units('Mpc*a/h'), units='Mpc a h**-1').in_units('kpc a h**-1')

In [282]:
pnb_sphere_centre

SimArray([151647.9 , 141852.69, 143663.56], dtype=float32, 'kpc')

In [283]:
pnb_sphere_radius

SimArray(322.09500122, 'kpc')

In [284]:
pnb_sphere = pnb.filt.Sphere(radius=pnb_sphere_radius, cen=pnb_sphere_centre)

In [285]:
pnb_sphere

Sphere(3.22e+02, array([151647.9 , 141852.69, 143663.56], dtype=float32))

In [286]:
pnb_sphere_gas = snap_dict['pnb']['snap'].gas[pnb_sphere]
# pnb_sphere_gas = pnb_sphere.where(snap_dict['pnb']['snap'].gas)

In [287]:
len(pnb_sphere_gas)

8007

### Manual Calculation

In [288]:
pnb_sphere_centre = np.repeat(np.array([sphere_centre]), repeats=len(snap_dict['pnb']['snap'].gas['pos']), axis=0)

In [289]:
pnb_sphere_centre

array([[151.6479 , 141.85269, 143.66356],
       [151.6479 , 141.85269, 143.66356],
       [151.6479 , 141.85269, 143.66356],
       ...,
       [151.6479 , 141.85269, 143.66356],
       [151.6479 , 141.85269, 143.66356],
       [151.6479 , 141.85269, 143.66356]],
      shape=(2243885, 3), dtype=float32)

In [290]:
pnb_radii_Mpc = euclidean_distance(snap_dict['pnb']['snap'].gas['pos'].in_units('Mpc'), pnb_sphere_centre)

In [291]:
np.shape(pnb_radii_Mpc)

(2243885, 1)

In [292]:
pnb_radii_Mpc

array([[ 7.821398 ],
       [ 7.8209414],
       [ 7.821242 ],
       ...,
       [19.56693  ],
       [19.56536  ],
       [19.566984 ]], shape=(2243885, 1), dtype=float32)

In [293]:
pnb_index_within_sphere = np.where(pnb_radii_Mpc <= sphere_radius)

In [294]:
pnb_index_within_sphere

(array([   5186,    5187,   27145, ..., 2234081, 2234082, 2234083],
       shape=(8006,)),
 array([0, 0, 0, ..., 0, 0, 0], shape=(8006,)))

In [295]:
pnb_gas_within_sphere_coords = pnb_radii_Mpc[pnb_radii_Mpc<=sphere_radius]

In [296]:
pnb_gas_within_sphere_coords

array([0.02535578, 0.03249368, 0.22816037, ..., 0.16704257, 0.16658902,
       0.16734165], shape=(8006,), dtype=float32)

In [297]:
len(pnb_gas_within_sphere_coords)

8006

# Number of particles in spheres

In [305]:
print(f'Number of gas particles in h5py sphere - manual calculation: {len(h5py_gas_within_sphere_coords)}')
print(f'Number of gas particles in yt sphere - manual calculation: {len(yt_gas_within_sphere_coords)}')
print(f"Number of gas particles in yt sphere - yt.snap.sphere: {len(yt_sphere['PartType0', 'ones'])}")
print(f'Number of gas particles in pynbody sphere - manual calculation: {len(pnb_gas_within_sphere_coords)}')
print(f'Number of gas particles in pynbody sphere - pynbody.filt.Sphere: {len(pnb_sphere_gas)}')

Number of gas particles in h5py sphere - manual calculation: 8006
Number of gas particles in yt sphere - manual calculation: 8006
Number of gas particles in yt sphere - yt.snap.sphere: 12175
Number of gas particles in pynbody sphere - manual calculation: 8006
Number of gas particles in pynbody sphere - pynbody.filt.Sphere: 8007
